In [2]:
#  In this mini-tutorial, several optimization problems are solved 
#  using the ModelingToolkit.jl and GalacticOptim.jl packages in Julia. 

#  The function to be optimized (minimized) is the Rosenbrock function:

#       (1 - x)^2 + 100 * (y - x^2)^2

#  The above function is a common test function for optimization methods. 
#  By inspection, it can be see that the minimum is at x = y = 1 
#  and the function value is zero at this location.  Despite having an
#  obvious solution, this problem is still a useful numerical test case.

#  The packages ModelingToolkit.jl, GlacticOptim.jl, and Optim.jl
#  need to be installed for this tutorial.

#  Date:  9/8/2021

#  Author:  Doug Frey, UMBC

#  Julia version 1.6.1 was used to create this tutorial

In [3]:
#  The following package versions were used 
#  for this tutorial:

#
#      Status `C:\Users\Douglas Frey\environment_v161_var1\Project.toml`
#   [a75be94c] GalacticOptim v2.0.3
#   [961ee093] ModelingToolkit v6.4.9
#   [429524aa] Optim v1.4.1
#   [1dea7af3] OrdinaryDiffEq v5.55.1
#   [91a5bcdd] Plots v1.21.3

#  Package versions can be checked by running the following commands.

#  First import Pkg:

       import Pkg

#  The command below is only needed if Julia 1.6.1 is being used in
#  a Jupyter notebook due to a bug in Julia 1.6.1. This command can
#  be deleted otherwise.

       Pkg.DEFAULT_IO[] = stdout

#  Write the status to the output:

       Pkg.status()

      Status `C:\Users\Douglas Frey\environment_v161_var1\Project.toml`
  [a75be94c] GalacticOptim v2.0.3
  [961ee093] ModelingToolkit v6.4.9
  [429524aa] Optim v1.4.1
  [1dea7af3] OrdinaryDiffEq v5.55.1
  [91a5bcdd] Plots v1.21.3


In [2]:
#  Make available for use the needed packages:

using ModelingToolkit, GalacticOptim, Optim

In [22]:
#  Example problem 1 where the Rosenbrock function
#  is minimized with no constraints

#   Newton Trust Region method will  be used

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Define the objective function

objective = (a - x)^2 + b * (y - x^2)^2

#  Create an OptmizationSystem object and name it

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

#  Set initial guess.  Note that on purpose a very bad 
#  initial guess will be used.

u0 = [ x => -1.0, y => -2.0]

#  Set parameters

params = [ a => 1.0, b => 100.0]

#  Define the problem to solve.  Note that the Newton Trust
#  Region method needs both the gradient and  Hessian matrix.
#  These will be determined by automatic differentiation (not
#  finite differences).

prob = OptimizationProblem(sys,u0, params, grad=true, hess=true)

#  Create a callback function to monitor the progress to convergence

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Use the Newton Trust Region method. Desired error is 10^-12. A maximum
#  of 100 iterations will be performed with objective function value 
#  reported every 5 iterations.  Maximum time limit is 100 seconds:

result = solve(prob, NewtonTrustRegion(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])


Objective function value =  -88.88888888888894
Objective function value =  480.3278688524589
Objective function value =  6493.2223543400305
Objective function value =  102494.16710456138
Objective function value =  1.6384942267850186e6
Objective function value =  2.621449422984221e7
Objective function value =  4.194304940571426e8
Objective function value =  6.710886449786324e9
Objective function value =  1.0737417111645421e11
Objective function value =  1.7179840057880579e12
Objective function value =  2.7487045060676395e13
 
Final value of objective function = -6.601367531975355e14
 
Final value for x = 3.6000000000001364
 


LoadError: BoundsError: attempt to access 1-element Vector{Float64} at index [2]

In [6]:
#  Now take a look at some results symbolically
#  to show symbolics ability of ModelingToolkit.jl

#  First show the gradient

Symbolics.gradient(objective,[x,y])

2-element Vector{Num}:
 2x - (2a) - (2b*x*(2y - (2(x^2))))
                  b*(2y - (2(x^2)))

In [7]:
#  Next show the Hessian matrix symbolically

Symbolics.hessian(objective,[x,y])

2×2 Matrix{Num}:
 2 + 8b*(x^2) - (2b*(2y - (2(x^2))))  -4b*x
                               -4b*x     2b

In [8]:
#  Next show d(objective_function)/dx symbolically

Dx = Differential(x)

expand_derivatives(Dx(objective))

2x - (2a) - (2b*x*(2y - (2(x^2))))

In [9]:
#  Example problem 2 where the Rosenbrock function
#  is minimized with no constraints

#   The Broyden, Fletcher, Golfarb, Shanno (BFGS) Quasi-Newton
#   method will be used

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Define the objective function

objective = (a - x)^2 + b * (y - x^2)^2

#  Create an OptmizationSystem object and name it

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

#  Set initial guess.  Note that on purpose a very bad 
#  initial guess will be used.

u0 = [ x => -1.0, y=>-2.0]

#  Set parameters

params = [ a => 1.0, b => 100.0]

#  Define the problem to solve.  Note that the BFGS method 
#  needs only the gradient to be supplied (not the Hessian matrix)
#  since the latter is approximated internally by the method.

prob = OptimizationProblem(sys,u0, params, grad=true, hess=false)

#  Create a callback function to monitor the progress to convergence

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Use the BFGS method. Desired error is 10^-12. A maximum
#  of 100 iterations will be performed with objective function value 
#  reported every 5 iterations.  Maximum time limit is 100 seconds:

result = solve(prob, BFGS(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])


Objective function value =  904.0
Objective function value =  2.741686784696231
Objective function value =  0.7183143047436891
Objective function value =  0.19150437378083918
Objective function value =  0.01045448959359955
Objective function value =  3.0853818626490574e-5
 
Final value of objective function = 1.5407439555097887e-30
 
Final value for x = 0.9999999999999994
 
Final value for y = 0.9999999999999988


In [10]:
#  Example problem 3 where the Rosenbrock function
#  is optimized with no constraints

#  The Nelder-Mead method will be used. Note this method
#  does not use either the gradient or Hessian and only
#  uses function evaluations.

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Remainder of statements are similar to the first two examples
#  so most comments are eliminated below.

objective = (a - x)^2 + b * (y - x^2)^2

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

u0 = [ x => -1.0, y=>-2.0]

params = [ a => 1.0, b => 100.0]

#  The Nelder-Mead method does not need either the gradient or
#  Hessian matrix.  Only function evaluations are used.

prob = OptimizationProblem(sys,u0, params, grad=false, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  The Nelder Mead method will be used.  Since this method is less
#  efficient per iteration, more iterations are needed than before.

result = solve(prob, NelderMead(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 20, time_limit = 100 , iterations=200)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])


Objective function value =  1584.0625
Objective function value =  1.1631774358416251
Objective function value =  0.29180164434997524
Objective function value =  0.04497657891310023
Objective function value =  4.900320352970663e-5
Objective function value =  1.387365135275737e-10
 
Final value of objective function = 8.195450959885444e-14
 
Final value for x = 1.0000001196491832
 
Final value for y = 1.0000002653057993


In [23]:
#  Example problem 4 where the Rosenbrock function
#  is optimized with upper and lower limits on the
#  search variables.

#  The Particle Swarm method will be used. Note this method
#  does not use either the gradient or Hessian and only
#  uses function evaluations. The method permits upper and
#  lower limits on the search variables which may be useful.
#  Particle Swarm method a global optimization and not a
#  local optimization like the other options shown above.

#  Define the variables and parameters.

@variables x y 
@parameters a b

#  Remainder of statements are similar to the first two examples
#  so most comments are eliminated below.

objective = (a - x)^2 + b * (y - x^2)^2

@named  sys = OptimizationSystem(objective, [x,y], [a,b])

u0 = [ x => -1.0, y=>-1.0]

params = [ a => 1.0, b => 100.0]

#  Niether gradient nor Hessian are needed

prob = OptimizationProblem(sys, u0, params, grad=false, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Define upper and lower limits for x and y.  First
#  element in the two vectors below corresponds to x and
#  the second element corresponds to y.

lower_limit = [0.0, 0.0]

upper_limit = [0.5, 0.5]

#  Particle Swarm method may needs lots of iterations.  100 search
#  particles will be used here.

         result = GalacticOptim.solve(prob, 
                Optim.ParticleSwarm(lower = lower_limit, upper = upper_limit, 
                   n_particles = 100), x_abstol = 1e-12, f_abstol = 1e-12, 
                      g_abstol = 1e-12, cb = callback, 
                      show_every = 10, time_limit = 600, iterations=100);

#  Note that due to the constraints the x=y=1 minimimum cannot be achieved.

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])

Objective function value =  4.672545911606741
Objective function value =  0.7571318506576622
Objective function value =  0.2572178731228865
Objective function value =  0.25021454245109676
Objective function value =  0.25000000221498797
Objective function value =  0.2500000007252524
Objective function value =  0.25000000000373196
Objective function value =  0.2500000000000003
Objective function value =  0.25
Objective function value =  0.25
Objective function value =  0.25
 
Final value of objective function = 0.25
 
Final value for x = 0.5
 
Final value for y = 0.2500000004420881


In [12]:
#  Example problem 5 where the Rosenbrock function
#  is optimized.  An equality constraint on x and y
#  will be used. The problem solved is:

#       minimize:   (1 - x)^2 + 100 * (y - x^2)^2
#       subject to:     x*y = 2.0

#  Using the concept of a penalty function, the above problem
#  can be solved as follows:

#    minimize (1-x)^2 + 100*(y-x^2)^2 + penalty_factor*(x*y-2.0)^2
#       where  penalty_factor is a large number (e.g., 10^6)

#  The BFGS method will be used. 

#  Define the variables and parameters.

@variables x y 
@parameters a b penalty_factor

#  Remainder of statements are similar to the first two examples
#  so most comments are eliminated below.

objective = (a - x)^2 + b * (y - x^2)^2  + penalty_factor*(x*y-2.0)^2

@named  sys = OptimizationSystem(objective, [x,y], [a,b,penalty_factor])

u0 = [ x => 1.0, y=> 1.0]

params = [ a => 1.0, b => 100.0, penalty_factor => 10^6]

prob = OptimizationProblem(sys,u0, params , grad=true, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Broyden, Fletcher, Golfarb, Shanno (BFGS) method will be used

result = solve(prob, BFGS(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

#  Determine error in equality constraint

error1 = result.minimizer[1]*result.minimizer[2] - 2.0

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final error in eq. constraint = ", error1)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])

Objective function value =  1.0e6
Objective function value =  33.64465731870688
Objective function value =  4.973376250606183
Objective function value =  0.08295360608528507
Objective function value =  0.06751169397671551
 
Final value of objective function = 0.06751169397671551
 
Final error in eq. constraint = -5.4549692318772713e-8
 
Final value for x = 1.2597392323717043
 
Final value for y = 1.5876301174528942


In [17]:
#  Example problem 6 where the Rosenbrock function
#  is optimized.  Example problem 2 will be repeated 
#  here except vector notation will be used. In particular
#  the problem involves the vectors w and par where w[1] = x
#  w[2] = y, par[1] = 1.0 and par[2] = 100.0

#  Define the variables and parameters

@variables w[1:2]  
@parameters par[1:2]

#  Define the objective function

objective = (par[1] - w[1])^2 + par[2] * (w[2] - w[1]^2)^2

#  Remainder of statements are similar to those above
#  so most comments are eliminated below.

 @named  sys = OptimizationSystem(objective, [w[1],w[2]], [par[1],par[2]])

w0 = [ w[1] => -1.0, w[2]=>-2.0]

params = [par[1] => 1.0, par[2] => 100.0]

prob = OptimizationProblem(sys, w0, params, grad=true, hess=false)

callback = function (p,l)
    println("Objective function value =  $l")
    return false
end

#  Broyden, Fletcher, Golfarb, Shanno (BFGS) method will be used

result = solve(prob, BFGS(),x_abstol = 1e-12, 
                   f_abstol = 1e-12, g_abstol = 1e-12, cb = callback, 
                    show_every = 5, time_limit = 100 , iterations=100)

println(" ")
println("Final value of objective function = ", result.minimum)
println(" ")
println("Final value for x = ", result.minimizer[1])
println(" ")
println("Final value for y = ", result.minimizer[2])

Objective function value =  904.0
Objective function value =  2.741686784696232
Objective function value =  0.7183143047436906
Objective function value =  0.19150437378082646
Objective function value =  0.010454489593595512
Objective function value =  3.085381863762898e-5
 
Final value of objective function = 1.5407439555097887e-30
 
Final value for x = 0.9999999999999994
 
Final value for y = 0.9999999999999988


In [18]:
#  Additional notes:

#  1.  If there are no  parameters in the problem,
#      then you can eliminate the @parameters statement.  But
#      you still  need to include an empty vector (i.e., [])
#      elsewhere where the parameters are supposed to 
#      appear, such as in the following statements:

#   @named  sys = OptimizationSystem(objective, [x,y], [])
#   prob = OptimizationProblem(sys,u0,[],grad=true,hess=false)

#  2.  If you want to solve a system of algebraic equations
#      such as:

#             f(x,y) = 0   and g(x,y) = 0

#      you can reformulate the above problem into the
#      equivalent optimization problem:

#           minimize:    (f(x,y))^2 + (g(x,y))^2

#      The above optimzation problem can be solved as shown
#      by the examples above.

# 3.   The penalty method can be extended to the case of an
#      inequality constraint.  For example, the following
#      problem:

#           minimize  g(x,y) 
#           subject to f(x,y) l.t.e 0.0

#      (where l.t.e. means "less than or equal to") is equivalent to

#         minimize  g(x,y) + penalty_factor * (min(0,f(x,y)))^2

#      where the output of the min function is the lower of the two input arguments.
